<a href="https://colab.research.google.com/github/ibadazam15-commits/LAB-3/blob/main/SPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Start Spark, create a SparkSession, and prepare the environment to run Spark jobs.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, upper, when, avg, count

# Start Spark session
spark = SparkSession.builder \
    .appName("ETL Pipeline") \
    .getOrCreate()

#Extract, transform, then Load in Spark (Similar manner as python Dataframe)"""

# -------------------------
# Extract
# -------------------------
spark_df = spark.read.csv("/content/drive/MyDrive/FDE/customers.csv", header=True, inferSchema=True)

# -------------------------
# TRANSFORM
# -------------------------

# Remove duplicates
spark_df = spark_df.dropDuplicates()

# Handle missing city values
spark_df = spark_df.fillna({"city": "Unknown"})

# Standardize customer names
spark_df = spark_df.withColumn("customer_name", upper(col("customer_name")))

# New Transformation: Spend Segmentation
spark_df = spark_df.withColumn(
    "spend_category",
    when(col("annual_spend") < 80000, "Low")
    .when((col("annual_spend") >= 80000) & (col("annual_spend") <= 120000), "Medium")
    .otherwise("High")
)

# Aggregation for analytics
elt_output = spark_df.groupBy("city", "spend_category").agg(
    count("customer_id").alias("total_customers"),
    avg("annual_spend").alias("avg_spend")
)

# -------------------------
# STORE ANALYTICS-READY DATA
# -------------------------
elt_output.write.mode("overwrite").parquet("elt_analytics_ready")

elt_output.show()



+---------+--------------+---------------+-----------------+
|     city|spend_category|total_customers|        avg_spend|
+---------+--------------+---------------+-----------------+
|  Unknown|          High|              1|         132000.0|
|  Unknown|           Low|              3|71333.33333333333|
|     Pune|          High|              1|         145000.0|
|    Noida|           Low|              1|          78000.0|
|  Kolkata|           Low|              1|          68000.0|
|Bengaluru|          High|              1|         150000.0|
|   Mumbai|          High|              1|         140000.0|
|  Kolkata|        Medium|              1|          90000.0|
|  Gurgaon|          High|              1|         155000.0|
|Ahmedabad|        Medium|              1|         110000.0|
|  Chennai|           Low|              1|          72000.0|
|Bengaluru|        Medium|              1|          82000.0|
|    Delhi|          High|              1|         170000.0|
|  Unknown|        Mediu